In [40]:
import json

with open('dataset_public.json') as dataset_file:
  data = json.load(dataset_file)

  titles = []
  headlines = []
  words = []
  texts = []
  for block in data:
    titles.append(block['title'])
    cur_headlines = []
    cur_wordmas = []
    cur_texts = []
    for header in block['news']:
      cur_headlines.append(header['headline'])
      cur_wordmas.append(str(header['headline']).split())
      cur_texts.append(str(header['body']))
    words.append(cur_wordmas)
    headlines.append(cur_headlines)
    texts.append(cur_texts)

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

for i in range(200):
    print(texts[i])
        

['Москва. 29 марта. ИНТЕРФАКС - Хроническим больным, в том числе с онкологией, после инфаркта и с высоким давлением можно прививаться от коронавируса COVID-19, в частности, вакциной "ЭпиВакКороной", если заболевание находится не в активной форме, сообщил вирусолог, иммунолог, руководитель екатеринбургского НИИ вирусных инфекций "Вектор" Роспотребнадзора Александр Семенов.\n"Безусловно. Если сейчас процесс у вас не активный, если вы в состоянии ремиссии, вне острого процесса, то не просто можно - нужно, потому что любая инфекция - она только ослабит ваш организм, и даст раку шансы на развитие", - сказал Семенов в эфире "Эхо Москвы" в понедельник, отвечая на соответствующий вопрос.\nАналогично он ответил на вопрос, можно ли делать прививку в возрасте за 60 лет после двух перенесенных инфарктов. "Если у человека сейчас все в порядке - давление под контролем, общие показатели нормальные, опять-таки - вне острого процесса, (...) если вы уже реабилитировались после инфаркта, то - пожалуйста,

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [65]:
import json
import re
from pprint import pprint
from string import punctuation

import numpy as np
from nltk import download, sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
from summa.keywords import keywords

# Загрузка стопслов nltk
download("stopwords")

MULTIPLE_WHITESPACE_PATTERN = re.compile(r"\s+", re.UNICODE)

for char in ['-', '/']:
    punctuation = punctuation.replace(char, '')


def normalizeWhitespace(text):
    return MULTIPLE_WHITESPACE_PATTERN.sub(replaceWhitespace, text)


def replaceWhitespace(match):
    text = match.group()

    if "\n" in text or "\r" in text:
        return "\n"
    else:
        return " "


def isBlank(string):
    return not string or string.isspace()


def getSymmetricMatrix(matrix):
    return matrix + matrix.T - np.diag(matrix.diagonal())


def coreCosineSimilarity(vector1, vector2):
    return 1 - cosine_distance(vector1, vector2)


class TextRank4Sentences():
    def __init__(self):
        self.damping = 0.85  # damping coefficient, usually is .85
        self.min_diff = 1e-5  # convergence threshold
        self.steps = 100  # iteration steps
        self.text_str = None
        self.sentences = None
        self.pr_vector = None

    def sentenceSimilarity(self, sent1, sent2, stopwords=None):
        if stopwords is None:
            stopwords = []

        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]

        all_words = list(set(sent1 + sent2))

        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)

        # build the vector for the first sentence
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1

        # build the vector for the second sentence
        for w in sent2:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1

        return coreCosineSimilarity(vector1, vector2)

    def buildSimilarityMatrix(self, sentences, stopwords=None):
        # create an empty similarity matrix
        sm = np.zeros([len(sentences), len(sentences)])

        for idx1 in range(len(sentences)):
            for idx2 in range(len(sentences)):
                if idx1 == idx2 or len(sentences[idx1]) < 7 or len(sentences[idx2]) < 7:
                    continue

                sm[idx1][idx2] = self.sentenceSimilarity(
                    sentences[idx1], sentences[idx2], stopwords=stopwords)

        # Get Symmeric matrix
        sm = getSymmetricMatrix(sm)

        # Normalize matrix by column
        norm = np.sum(sm, axis=0)
        # this is to ignore the 0 element in norm
        sm_norm = np.divide(sm, norm, where=norm != 0)

        return sm_norm

    def runPageRank(self, similarity_matrix):

        pr_vector = np.array([1] * len(similarity_matrix))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr_vector = (1 - self.damping) + self.damping * \
                np.matmul(similarity_matrix, pr_vector)
            if abs(previous_pr - sum(pr_vector)) < self.min_diff:
                break
            else:
                previous_pr = sum(pr_vector)

        return pr_vector

    def getSentence(self, index):

        try:
            return self.sentences[index]
        except IndexError:
            return ""

    def getTopSentences(self, number=1):
        top_sentences = {}

        if self.pr_vector is not None:

            sorted_pr = np.argsort(self.pr_vector)
            sorted_pr = list(sorted_pr)
            sorted_pr.reverse()
            print(len(self.sentences))
            index = 0
            for epoch in range(number):
                print(str(sorted_pr[index]) + " : " +
                      str(self.pr_vector[sorted_pr[index]]))
                sent = self.sentences[sorted_pr[index]]
                sent = normalizeWhitespace(sent)
                top_sentences[sent] = self.pr_vector[sorted_pr[index]]
                index += 1

        return top_sentences

    def analyze(self, text, stop_words=None):
        self.text_str = text
        self.sentences = sent_tokenize(self.text_str)

        tokenized_sentences = [word_tokenize(sent) for sent in self.sentences]

        similarity_matrix = self.buildSimilarityMatrix(
            tokenized_sentences, stop_words)

        self.pr_vector = self.runPageRank(similarity_matrix)
        print(self.pr_vector)


STOP_WORDS = open('stopWords.txt', 'r').readlines()

#data = json.loads(open('../data/dataset_public.json', encoding='utf-8').read())


def generateSummarizedText(texts, sentenceNumber=1, stopWords=STOP_WORDS):
    summarizedText = ''

    for text in texts:
        tr4sh = TextRank4Sentences()
        tr4sh.analyze(text, stopWords)
        summarizedText += '\n'.join(tr4sh.getTopSentences(sentenceNumber).keys())

    return summarizedText


#texts = []
#for entry in data[0]['news'][:10]:
#    texts.append(entry['body'])


mystem = Mystem()
russian_stopwords = stopwords.words("russian")


def preprocessText(text):
    text = re.sub(
        r' +', ' ', text.translate(str.maketrans('', '', punctuation)))
    tokens = mystem.lemmatize(text.lower())
    tokens = [
        token for token in tokens if token not in russian_stopwords and token != ' ' and token not in punctuation]
    text = ' '.join(list(set(tokens)))

    return text

# text = preprocessText(texts[0][0])
# print(text)
# print(headlines[0])
# summarizedText = generateSummarizedText(headlines[0])
# print(summarizedText)
# print(preprocessText(summarizedText))
# print('ok')

print(texts[0][0])
keys = keywords(preprocessText(texts[0][0]), language='russian', words=10).split()


morph = pymorphy2.MorphAnalyzer(lang='ru')
final_set = []
for word in keys:
    cur_mas = morph.parse(word)[0]
    if cur_mas.tag.POS == 'NOUN':
        final_set.append(word)

print(final_set)
    


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sestepanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Москва. 29 марта. ИНТЕРФАКС - Хроническим больным, в том числе с онкологией, после инфаркта и с высоким давлением можно прививаться от коронавируса COVID-19, в частности, вакциной "ЭпиВакКороной", если заболевание находится не в активной форме, сообщил вирусолог, иммунолог, руководитель екатеринбургского НИИ вирусных инфекций "Вектор" Роспотребнадзора Александр Семенов.
"Безусловно. Если сейчас процесс у вас не активный, если вы в состоянии ремиссии, вне острого процесса, то не просто можно - нужно, потому что любая инфекция - она только ослабит ваш организм, и даст раку шансы на развитие", - сказал Семенов в эфире "Эхо Москвы" в понедельник, отвечая на соответствующий вопрос.
Аналогично он ответил на вопрос, можно ли делать прививку в возрасте за 60 лет после двух перенесенных инфарктов. "Если у человека сейчас все в порядке - давление под контролем, общие показатели нормальные, опять-таки - вне острого процесса, (...) если вы уже реабилитировались после инфаркта, то - пожалуйста, при

In [188]:
from math import sqrt
import copy
def return_top_k_phrases(list_of_phrases, text, k):
    morph = pymorphy2.MorphAnalyzer(lang='ru')
    
    text = re.sub(
        r' +', ' ', text.translate(str.maketrans('', '', punctuation)))
    parsed_text = text.split()
    parsed_named_text = []
    for word in parsed_text:
        cur_mas = morph.parse(word)[0]
        cur_pos = cur_mas.tag.POS
        cur_pos = str(cur_mas.tag.POS)
        if cur_pos == 'ADJV':
            cur_pos = 'ADJ'
        if cur_pos == 'ADJS':
            cur_pos = 'ADJ'
        elif cur_pos != 'VERB' and cur_pos != 'NOUN':
            continue
        parsed_named_text.append(str(cur_mas.normal_form) + '_' + str(cur_pos))
    
    
    result_vec = []
    for phrase in list_of_phrases:
        r_phrase = copy.deepcopy(phrase)
        r_phrase = re.sub(
        r' +', ' ', r_phrase.translate(str.maketrans('', '', punctuation)))
        cur_points = 0.0
        words = r_phrase.split()
        size = len(words)
        for word in words:
            cur_mas = morph.parse(word)[0]
            cur_pos = str(cur_mas.tag.POS)
            if cur_pos == 'ADJV':
                cur_pos = 'ADJ'
            if cur_pos == 'ADJS':
                cur_pos = 'ADJ'
            if cur_pos == 'INFN':
                cur_pos = 'VERB'
            elif cur_pos != 'VERB' and cur_pos != 'NOUN':
                size-=1
                continue
            parsed_word = str(cur_mas.normal_form) + '_' + str(cur_pos)
            cur_metric = 0.0
            for wordt in parsed_named_text:
                try:
                    cm = (russian_model.similarity(parsed_word, wordt))
                except:
                    cm = 0.1
                cur_metric += (cm)**2
            cur_points+=(sqrt(cur_metric/len(parsed_named_text)))
        if size != 0:
            result_vec.append((cur_points/size, phrase))
        else:
            result_vec.append((0,phrase))
    result_vec.sort()
    print(result_vec)

In [ ]:
import gensim
import gensim.downloader as download_api
russian_model = download_api.load('word2vec-ruscorpora-300')
print('ok')

ok


In [189]:
text = texts[0][0]
rand_shit = 'особенность ковивак эхо план проблема'
real_title = headlines[0][0]

mas = [rand_shit, real_title]
return_top_k_phrases(mas, text, 1)

[(0.18216140146650545, 'особенность ковивак эхо план проблема'), (0.23923956756130074, 'Онкология, диабет и инфаркт не являются противопоказанием против вакцинации от COVID-19 - глава екатеринбургского НИИ "Вектор"')]


In [183]:
russian_model.most_similar('начать_VERB')

russian_model.similarity('особенность_NOUN', 'начаться_VERB')

KeyError: "Key 'начать_VERB' not present"

In [180]:
morph = pymorphy2.MorphAnalyzer(lang='ru')
cur_mas = morph.parse('начаться')[0]
print(cur_mas.tag.POS)

INFN
